In [ ]:
pip install beautifulsoup4 requests
pip install lxml

In [3]:
import os
from time import sleep
import tqdm

import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup

In [5]:
# CSV 파일 로드
df = pd.read_csv('../DATA/job.csv')
mini_df = df.head(3)
print("CSV load complete")

# 결과를 저장할 DataFrame 초기화
result_df = pd.DataFrame(columns=['keyword', 'base64_data'])

# START!
for _ in df['English']:
    KEYWORD = _
    URL = "https://google.com/search?q="+KEYWORD+"&tbm=isch"

    # 웹 드라이버 생성
    driver = webdriver.Chrome()

    # 링크 열기
    driver.get(URL)
    driver.fullscreen_window()

    sleep(1.5)

    # 페이지의 HTML 가져오기
    html = driver.page_source

    # BeautifulSoup으로 파싱
    soup = BeautifulSoup(html, 'lxml')

    # 클래스 이름을 사용하여 모든 이미지 태그를 선택합니다.
    img_tags = soup.find_all('img', class_='rg_i Q4LuWd')

    # 각 이미지의 Base64 데이터를 임시 DataFrame에 저장합니다.
    temp_data = []
    for img_tag in img_tags:
        if 'src' in img_tag.attrs:
            base64_data = img_tag['src']
            temp_data.append({'keyword': KEYWORD, 'base64_data': base64_data})

    temp_df = pd.DataFrame(temp_data)
    result_df = pd.concat([result_df, temp_df], ignore_index=True)

    # 스크린샷 찍기
    directory_path = '../DATA/screenshot/' + KEYWORD 
    screenshot_path = directory_path + '/job_'+KEYWORD+'_img00.png'
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    driver.save_screenshot(screenshot_path)

    # 페이지 종료
    driver.quit()
# 결과 출력
print(result_df)

# 결과를 CSV 파일로 저장 (선택사항)
result_df.to_csv('../DATA/result_base64_data.csv', index=False)
print("DONE!")

CSV load complete
                 keyword                                        base64_data
0     Software Developer  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
1     Software Developer  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
2     Software Developer  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
3     Software Developer  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
4     Software Developer  data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
...                  ...                                                ...
2034         Interpreter  https://encrypted-tbn0.gstatic.com/images?q=tb...
2035         Interpreter  https://encrypted-tbn0.gstatic.com/images?q=tb...
2036         Interpreter  https://encrypted-tbn0.gstatic.com/images?q=tb...
2037         Interpreter  https://encrypted-tbn0.gstatic.com/images?q=tb...
2038         Interpreter  https://encrypted-tbn0.gstatic.com/images?q=tb...

[2039 rows x 2 columns]
DONE!
